In [ ]:
{
  "cells": [
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "setup_env"
      },
      "outputs": [],
      "source": [
        "!pip install stable-baselines3 gym pandas numpy matplotlib shimmy mlflow prometheus_client snakemake\n",
        "\n",
        "# Télécharger les scripts depuis le dépôt GitHub\n",
        "!wget https://raw.githubusercontent.com/<ton-utilisateur>/<ton-repo>/main/BESSBatteryEnv.py\n",
        "!wget https://raw.githubusercontent.com/<ton-utilisateur>/<ton-repo>/main/train_ppo.py\n",
        "!wget https://raw.githubusercontent.com/<ton-utilisateur>/<ton-repo>/main/evaluate_ppo.py\n",
        "\n",
        "# Créer le Snakefile\n",
        "with open('Snakefile', 'w') as f:\n",
        "    f.write(\"\"\"\n",
        "rule all:\n",
        "    input:\n",
        "        '/content/drive/MyDrive/ppo_bess_model.zip',\n",
        "        '/content/drive/MyDrive/evaluation_metrics.json'\n",
        "\n",
        "rule train_ppo:\n",
        "    output:\n",
        "        '/content/drive/MyDrive/ppo_bess_model.zip'\n",
        "    shell:\n",
        "        'python train_ppo.py'\n",
        "\n",
        "rule evaluate_ppo:\n",
        "    input:\n",
        "        '/content/drive/MyDrive/ppo_bess_model.zip'\n",
        "    output:\n",
        "        '/content/drive/MyDrive/evaluation_metrics.json'\n",
        "    shell:\n",
        "        'python evaluate_ppo.py'\n",
        "\"\"\")\n",
        "\n",
        "# Exécuter Snakemake\n",
        "!snakemake -s Snakefile --cores 1\n",
        "\n",
        "# Pousser les métriques vers Prometheus/PushGateway\n",
        "import json\n",
        "import os\n",
        "\n",
        "PUSHGATEWAY_URL = os.getenv('PUSHGATEWAY_URL', 'https://e546-160-179-231-239.ngrok-free.app')\n",
        "\n",
        "# Charger les métriques d'entraînement\n",
        "with open('/content/drive/MyDrive/ppo_bess_model_metrics.json', 'r') as f:\n",
        "    train_metrics = json.load(f)\n",
        "for key, value in train_metrics.items():\n",
        "    !echo \"{key}{{run=\\\"ppo_training\\\"}} {value}\" | curl --data-binary @- $PUSHGATEWAY_URL/metrics/job/ppo_metrics\n",
        "\n",
        "# Charger les métriques d'évaluation\n",
        "with open('/content/drive/MyDrive/evaluation_metrics.json', 'r') as f:\n",
        "    eval_metrics = json.load(f)\n",
        "for key, value in eval_metrics.items():\n",
        "    !echo \"{key}{{run=\\\"ppo_evaluation\\\"}} {value}\" | curl --data-binary @- $PUSHGATEWAY_URL/metrics/job/ppo_metrics\n",
        "\n",
        "# Pousser une métrique de succès\n",
        "!echo \"success 1\" | curl --data-binary @- $PUSHGATEWAY_URL/metrics/job/ppo_metrics\n"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.9.5"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}